1. 调用必要模块，定义必要常数

In [1]:
from PIL import Image
import numpy as np
import time
DIMENSION = 256
PIC_NUM = 100
SIGMA = 1

2. 获取所有图片的RGB矩阵

In [2]:
start_time = time.time()
RED_init = []
GREEN_init = []
BLUE_init = []
for i in range(PIC_NUM):
    if i < 10:
        i = str(i)
        img_fn = "Images/agricultural/agricultural0" + i + ".tif"
    else:
        i = str(i)
        img_fn = "Images/agricultural/agricultural" + i + ".tif"

    img = Image.open(img_fn)
    rgb_img = img.convert("RGB")
    im = np.array(rgb_img)
    Red = im[:,:,0]
    Green = im[:,:,1]
    Blue = im[:,:,2]
    RED_init.append(Red)
    GREEN_init.append(Green)
    BLUE_init.append(Blue)

# 转化为numpy列表
RED_init = np.array(RED_init) # 存储每张图片的R矩阵
GREEN_init = np.array(GREEN_init) # 存储每张图片的G矩阵
BLUE_init = np.array(BLUE_init) # 存储每张图片的B矩阵

3. 将所有的矩阵放在一起，构造100 * 196608的矩阵

In [3]:
pictures = np.hstack((RED_init[0].flatten(), GREEN_init[0].flatten(), BLUE_init[0].flatten()))
for i in range(1, PIC_NUM):
    pic_data = np.hstack((RED_init[i].flatten(), GREEN_init[i].flatten(), BLUE_init[i].flatten()))
    pictures = np.row_stack((pictures, pic_data))
picture_mean = np.mean(pictures, axis = 0)

4. 构造高斯核函数

In [4]:
def kernel(x, y, sigma):
    norm = np.dot((x - y).T, (x - y))
    result = np.exp(-(norm / 2 / sigma ** 2)) # 高斯核函数
    return result

5. 编写一个可以求矩阵特征值与特征向量的函数（这里使用了论文中的QR分解方法）

In [5]:
def eigenvalue_func(cov):
    A_pre = np.zeros((PIC_NUM, PIC_NUM), dtype=float)
    A_post = cov
    S = np.eye(PIC_NUM, dtype=float)
    while np.linalg.norm(np.diag(A_post) - np.diag(A_pre)) >= 0.00001:
        A_pre = A_post
        Q, R = np.linalg.qr(A_post)
        S = np.matmul(S, Q)
        A_post = np.matmul(R, Q)
    return np.diag(A_post), S

6. 压缩一张图片

In [6]:
KernelMatrix = np.zeros((PIC_NUM, PIC_NUM))
for i in range(PIC_NUM):
    for j in range(PIC_NUM):
        KernelMatrix[i][j] = kernel(pictures[i], pictures[j], SIGMA)

In [7]:
I = np.ones((PIC_NUM, PIC_NUM)) / PIC_NUM
Normalized_K = KernelMatrix - I @ KernelMatrix - KernelMatrix @ I + I @ KernelMatrix @ I

In [8]:
eigenvalue, eigenvector = eigenvalue_func(Normalized_K)
Normalized_eigenvector = np.zeros((PIC_NUM, PIC_NUM))
for i in range(PIC_NUM):
    if eigenvalue[i] < 0:
        break
    
    norm = np.sqrt(np.dot(eigenvector[:, i].T, eigenvector[:, i]))
    Normalized_eigenvector[:, i] = eigenvector[:, i] / norm / np.sqrt(eigenvalue[i])

In [9]:
k = 100
project_vector = Normalized_K @ Normalized_eigenvector[:, :k]
new_axis = np.matmul(project_vector.T, pictures)

# reconstruct the image data with k PCs
new_pictures = np.matmul(project_vector[:, :k], new_axis[:k, :])
im3 = np.matmul(project_vector[:, :k], new_axis[:k, :])[0]

im3 = im3.astype('uint8')

# reconstruct the three (R,G,B) channels
im3_channels = np.hsplit(im3, 3)
for i in range(3):
    im3_channels[i] = im3_channels[i].reshape((256, 256))
im4 = np.zeros((256, 256, 3))
for i in range(3):
    im4[:,:,i] = im3_channels[i]
new_image = Image.fromarray(np.uint8(im4*255*255))
new_image.save("./test1.tif")
end_time = time.time()

7. 计算agricultural0.tif图像的重构误差

In [10]:
Red_Error = RED_init[0] - im3_channels[0]
Green_Error = GREEN_init[0] - im3_channels[1]
Blue_Error = BLUE_init[0] - im3_channels[2]
Error = np.linalg.norm(Red_Error, ord="fro") + np.linalg.norm(Green_Error, ord="fro") + np.linalg.norm(Blue_Error, ord="fro")
print(Error)
print("Running time:", end_time - start_time, "seconds")

96528.65310627266
Running time: 4.590271472930908 seconds
